In [ ]:
!pip install -q transformers datasets
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

ds = load_dataset("ErfanMoosaviMonazzah/Task-Dialogue-FewShot-Dataset")
tokenizer = AutoTokenizer.from_pretrained('ErfanMoosaviMonazzah/T5-Task-Dialogue-Pretrained')

# splits = ['attraction', 'hotel', 'laptop', 'restaurant', 'taxi', 'train', 'tv']
splits = ['train', 'tv', 'laptop']

Extracting data files:   0%|          | 0/14 [00:00<?, ?it/s]

Generating attraction.test split:   0%|          | 0/340 [00:00<?, ? examples/s]

Generating attraction.train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating hotel.test split:   0%|          | 0/78 [00:00<?, ? examples/s]

Generating hotel.train split:   0%|          | 0/51 [00:00<?, ? examples/s]

...

Generating train.train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating tv.test split:   0%|          | 0/680 [00:00<?, ? examples/s]

Generating tv.train split:   0%|          | 0/51 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ErfanMoosaviMonazzah___parquet/ErfanMoosaviMonazzah--Task-Dialogue-FewShot-Dataset-9fd196ff8506ad68/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
MAX_INPUT_LEN = 512
MAX_OUTPUT_LEN = 128
NUM_EPOCHS = 20
LR = 1e-5
K = 3 # num_samples
P = 0.9 # top_p
LENGTH = 128 # length

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import json

for split in splits:
    print(f'---- Generating for {split.capitalize()} Tasks ----')
    inputs = ds[f'{split}.test']['text']
    model_id = f'ErfanMoosaviMonazzah/T5-Task-Dialogue-FineTuned-{split.capitalize()}-{NUM_EPOCHS}-{LR}'

    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    si = 0
    while si < len(inputs):
        if len(inputs) - si <= 300:
            ei = len(inputs)
        else:
            ei = si + 300

        input_ids = tokenizer.batch_encode_plus(inputs[si:ei], return_tensors="pt", padding=True)["input_ids"]
        output_ids = model.generate(input_ids, top_p=P, do_sample=True, num_return_sequences=K)

        print("output_ids generated")

        outputs = []
        tmp_l = []
        for i, output_id in enumerate(output_ids):
            if i % K == 0:
                tmp_l = []

            t_out = ' & ' + tokenizer.decode(output_id, skip_special_tokens=True)+'|endoftext|'
            t_out = t_out + (LENGTH - len(t_out)) * '!'
            tmp_l.append(t_out)
            if i%K == K-1:
                outputs.append(tmp_l)

        with open(f'resutls-{model_id.split("/")[1]}-{si//300}.json', 'w') as jf:
            json.dump(outputs, jf, indent = 4)

        si = ei


---- Generating for Train Tasks ----


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


output_ids generated


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


output_ids generated
output_ids generated
---- Generating for Tv Tasks ----


output_ids generated
output_ids generated
output_ids generated
---- Generating for Laptop Tasks ----


output_ids generated
output_ids generated
output_ids generated
output_ids generated
output_ids generated


In [ ]:
from os import scandir
import zipfile

res_paths = []
with scandir('.') as entries:
    for file in entries:
        if file.name.startswith('results'):
            res_paths.append(file.path)



with zipfile.ZipFile('all_results.zip', 'w') as zipMe:
    for file in res_paths:
        zipMe.write(file, compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
assert False

In [ ]:
with open('test.txt', 'r') as file:
    lines = file.readlines()

In [ ]:
inputs = [line.split('&')[0].strip() for line in lines]
outputs = [line.split('&')[1].strip() for line in lines]

In [ ]:
generates= pipe(inputs)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("ErfanMoosaviMonazzah/T5-Task-Dialogue-Pretrained")
tokenizer = AutoTokenizer.from_pretrained("ErfanMoosaviMonazzah/T5-Task-Dialogue-Pretrained")

In [ ]:
k = 2
input_ids = tokenizer.batch_encode_plus(inputs, return_tensors="pt", padding=True)["input_ids"]
output_ids = model.generate(input_ids, top_p=0.9, do_sample=True, num_return_sequences=k)
# outputs = [' & ' + tokenizer.decode(output_id, skip_special_tokens=True) + len(tokenizer.decode(output_id, skip_special_tokens=True)) for output_id in output_ids]

outputs = []
tmp_l = []
for i, output_id in enumerate(output_ids):
    if i % k ==0:
        tmp_l = []

    t_out = ' & ' + tokenizer.decode(output_id, skip_special_tokens=True)+'|endoftext|'
    t_out = t_out + (80 - len(t_out)) * '!'
    tmp_l.append(t_out)
    if i%k == k-1:
        outputs.append(tmp_l)



In [ ]:
outputs

In [ ]:
import json

with open('resutls.json', 'w') as jf:
    json.dump(outputs, jf, indent = 4)
